In [1]:
import numpy as np
import re

# ------------------- INPUT PARSER -------------------
def load_input(filename):
    shapes, regions = {}, []
    region_re = re.compile(r'^(\d+)x(\d+):')
    idx, buff, parsing_shapes = None, [], True

    for line in open(filename):
        s = line.strip()
        if not s: continue
        if region_re.match(s): parsing_shapes=False
        if parsing_shapes:
            if s.endswith(":"):
                if idx is not None: shapes[idx]=buff
                idx=int(s[:-1]); buff=[]
            else: buff.append(s)
        else:
            m=region_re.match(s)
            if not m: continue
            W,H=int(m.group(1)),int(m.group(2))
            qty=list(map(int,line.split(":",1)[1].split()))
            regions.append((W,H,qty))
    if idx is not None: shapes[idx]=buff
    return shapes, regions

# ------------------- SHAPES ORIENTATIONS -------------------
def normalize(cells):
    min_r=min(r for r,_ in cells); min_c=min(c for _,c in cells)
    return tuple(sorted((r-min_r,c-min_c) for r,c in cells))

def rotate(cells): return normalize([(c,-r) for r,c in cells])
def flip(cells): return normalize([(r,-c) for r,c in cells])

def all_orients(grid):
    cells=[(r,c) for r,row in enumerate(grid) for c,v in enumerate(row) if v=="#"]
    base=normalize(cells); seen=set(); out=[]; cur=base
    for _ in range(4):
        cur=rotate(cur)
        if cur not in seen: seen.add(cur); out.append(cur)
        f=flip(cur)
        if f not in seen: seen.add(f); out.append(f)
    return out

# ------------------- BACKTRACKING -------------------
def can_place(board, orient, r_off, c_off):
    for r,c in orient:
        rr,cc=r+r_off,c+c_off
        if rr>=board.shape[0] or cc>=board.shape[1] or board[rr,cc]: return False
    return True

def place(board, orient, r_off, c_off, val):
    for r,c in orient: board[r+r_off,c+c_off]=val

def dfs(board, shapes_order, orient_data, idx):
    if idx==len(shapes_order): return True
    sid=shapes_order[idx]
    for orient in orient_data[sid]:
        max_r=max(r for r,_ in orient); max_c=max(c for _,c in orient)
        for r_off in range(board.shape[0]-max_r):
            for c_off in range(board.shape[1]-max_c):
                if can_place(board, orient, r_off, c_off):
                    place(board, orient, r_off, c_off, 1)
                    if dfs(board, shapes_order, orient_data, idx+1): return True
                    place(board, orient, r_off, c_off, 0)
    return False

def solve_region(W,H,qtys,orient_data):
    # Quick pre-check: total area
    total_cells = sum(q*len(next(iter(orient_data[i]))) for i,q in enumerate(qtys) if q>0)
    if total_cells > W*H: return False
    # Quick pre-check: bounding box
    for i,q in enumerate(qtys):
        if q==0: continue
        max_r=max(r for orient in orient_data[i] for r,_ in orient)
        max_c=max(c for orient in orient_data[i] for _,c in orient)
        if max_r>=H or max_c>=W: return False
    # Prepare order
    shapes_order=[]
    for sid, q in enumerate(qtys):
        shapes_order+=[sid]*q
    board=np.zeros((H,W),dtype=np.uint8)
    return dfs(board, shapes_order, orient_data, 0)

# ------------------- MAIN -------------------
def main():
    shapes_raw, regions = load_input("input.txt")
    orient_data={sid: all_orients(grid) for sid,grid in shapes_raw.items()}
    total_fit=0
    for W,H,qtys in regions:
        if solve_region(W,H,qtys,orient_data): total_fit+=1
    print(total_fit)

if __name__=="__main__":
    main()


490
